In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp

In [2]:
# input parameters
factory = ['Detroit', 'Los Angeles', 'Austin']      # list of factories (sources)
customer = ['Just Sports', 'Sports N Stuff', 'Sports Dude']  # list of customers (destinations)
dc = ['Iowa','Maryland', 'Idaho', 'Arkansas']                # list of distribution centers (transship)
supply = [350, 350, 700]      # units available at each factory
demand = [200, 500, 650]  # units required at each customer
cost_in = [[25.00, 25.00, 35.00, 40.00],     # inbound shipment cost from Detroit to Iowa, Maryland, Idaho, and Arkansas                
           [35.00, 45.00, 35.00, 42.50],     # from Los Angeles
           [40.00, 40.00, 42.50, 32.50]]     # from Austin
cost_out = [[30.00, 17.50, 30.00],  # outbound shipment cost from Iowa to Just Sports, Sports N Stuff, and Sports Dude
            [20.00, 32.50, 40.00],  # from Maryland
            [35.00, 40.00, 32.50],  # from Idaho
            [27.50, 25.00, 42.50]]  # from Arkansas
dc_cap = 500

In [3]:
# initialize LP model object
mymodel = glp.Solver('Sports', glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [4]:
# create two dimensional lists of shipment variables
ship_in = list(range(len(factory)))
for i in range(len(factory)):
    ship_in[i] = list(range(len(dc)))
    for j in range(len(dc)):
        ship_in[i][j] = mymodel.NumVar(0, mymodel.infinity(), factory[i]+dc[j])

In [5]:
ship_out = list(range(len(dc)))
for j in range(len(dc)):
    ship_out[j] = list(range(len(customer)))
    for k in range(len(customer)):
        ship_out[j][k] = mymodel.NumVar(0, mymodel.infinity(), dc[j]+customer[k])

In [6]:
# define objective function
TotCost = mymodel.Objective()
TotCost.SetMinimization()
# inbound shipment costs:
for i in range(len(factory)):
    for j in range(len(dc)):
        TotCost.SetCoefficient(ship_in[i][j], cost_in[i][j])
# outbound shipment costs:
for j in range(len(dc)):
    for k in range(len(customer)):
        TotCost.SetCoefficient(ship_out[j][k], cost_out[j][k])

In [7]:
# define supplier supply constraints
supply_constr = list(range(len(factory)))
for i in range(len(factory)):
    supply_constr[i] = mymodel.Constraint(0, supply[i])
    for j in range(len(dc)):
        supply_constr[i].SetCoefficient(ship_in[i][j], 1)

In [8]:
# define customer demand constraints
demand_constr = list(range(len(customer)))
for k in range(len(customer)):
    demand_constr[k] = mymodel.Constraint(demand[k], demand[k])
    for j in range(len(dc)):
        demand_constr[k].SetCoefficient(ship_out[j][k], 1)

In [9]:
# define D.C processing capacity constraints
in_processing_constr = list(range(len(dc)))
for j in range(len(dc)):
    in_processing_constr[j] = mymodel.Constraint(0,dc_cap)
    for i in range(len(factory)):
        in_processing_constr[j].SetCoefficient(ship_in[i][j], 1)
out_processing_constr = list(range(len(dc)))
for j in range(len(dc)):
    out_processing_constr[j] = mymodel.Constraint(0,dc_cap)
    for k in range(len(customer)):
        out_processing_constr[j].SetCoefficient(ship_out[j][k], 1)

In [10]:
# define flow constraints
flow_constr = list(range(len(dc)))
for j in range(len(dc)):
    flow_constr[j] = mymodel.Constraint(0,0)
    for i in range(len(factory)):
        flow_constr[j].SetCoefficient(ship_in[i][j], 1)
    for k in range(len(customer)):
        flow_constr[j].SetCoefficient(ship_out[j][k], -1)

In [11]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Total Cost = %.2f' % TotCost.Value())

Solution Status = 0
Number of variables = 24
Number of constraints = 18
Optimal Solution:
Total Cost = 79625.00


In [12]:
# Display the shipments
print ('Shipments from manufacturers to suppliers')
for i in range(len(factory)):
    print(factory[i])
    for j in range(len(dc)):
        print('   %10s \t %5.1f' % (dc[j], ship_in[i][j].solution_value()))
        
print ('Shipments from DCs to customers')       
for j in range(len(dc)):
    print(dc[j])
    for k in range(len(customer)):
        print('   %10s \t %5.1f' % (customer[k], ship_out[j][k].solution_value()))

Shipments from manufacturers to suppliers
Detroit
         Iowa 	 300.0
     Maryland 	  50.0
        Idaho 	   0.0
     Arkansas 	   0.0
Los Angeles
         Iowa 	 200.0
     Maryland 	   0.0
        Idaho 	 150.0
     Arkansas 	   0.0
Austin
         Iowa 	   0.0
     Maryland 	 150.0
        Idaho 	   0.0
     Arkansas 	 500.0
Shipments from DCs to customers
Iowa
   Just Sports 	   0.0
   Sports N Stuff 	   0.0
   Sports Dude 	 500.0
Maryland
   Just Sports 	 200.0
   Sports N Stuff 	   0.0
   Sports Dude 	   0.0
Idaho
   Just Sports 	   0.0
   Sports N Stuff 	   0.0
   Sports Dude 	 150.0
Arkansas
   Just Sports 	   0.0
   Sports N Stuff 	 500.0
   Sports Dude 	   0.0


In [13]:
# display supply constraint Information
print('Factory \t LHS \t RHS \t Slack \t Dual')
LHS = mymodel.ComputeConstraintActivities()
for i in range(len(factory)):
    slack = supply[i] - LHS[i]
    print('%11s \t %3.1f \t %3.1f \t %4.1f \t %5.2f' % (factory[i], LHS[i], supply[i], slack, supply_constr[i].dual_value()))

Factory 	 LHS 	 RHS 	 Slack 	 Dual
    Detroit 	 350.0 	 350.0 	  0.0 	 -15.00
Los Angeles 	 350.0 	 350.0 	  0.0 	 -5.00
     Austin 	 650.0 	 700.0 	 50.0 	  0.00


In [14]:
# display demand constraint Information
print('Customer \t LHS \t RHS \t Slack \t Dual')
for k in range(len(customer)):
    slack = LHS[k + len(factory)] - demand[k]
    print('%10s \t %3.1f \t %3.1f \t %2.1f \t %5.2f' % (customer[k], LHS[k + len(factory)], demand[k], slack, demand_constr[k].dual_value()))

Customer 	 LHS 	 RHS 	 Slack 	 Dual
Just Sports 	 200.0 	 200.0 	 0.0 	 60.00
Sports N Stuff 	 500.0 	 500.0 	 0.0 	 57.50
Sports Dude 	 650.0 	 650.0 	 0.0 	 72.50


In [15]:
# display capacity constraint Information
print('DC \t\t LHS \t RHS \t Slack \t Dual')
for j in range(len(dc)):
    slack = dc_cap - LHS[j + len(factory) + len(customer)]
    print('%10s \t %3.1f \t %3.1f \t %2.1f \t %5.2f' % (dc[j], LHS[j + len(factory) + len(customer)], dc_cap, slack, in_processing_constr[j].dual_value()))
print('DC \t\t LHS \t RHS \t Slack \t Dual')
for j in range(len(dc)):
    slack = dc_cap - LHS[j + len(factory) + len(customer)+4]
    print('%10s \t %3.1f \t %3.1f \t %2.1f \t %5.2f' % (dc[j], LHS[j + len(factory) + len(customer)], dc_cap, slack, out_processing_constr[j].dual_value()))

DC 		 LHS 	 RHS 	 Slack 	 Dual
      Iowa 	 500.0 	 500.0 	 0.0 	 -2.50
  Maryland 	 200.0 	 500.0 	 300.0 	  0.00
     Idaho 	 150.0 	 500.0 	 350.0 	  0.00
  Arkansas 	 500.0 	 500.0 	 0.0 	  0.00
DC 		 LHS 	 RHS 	 Slack 	 Dual
      Iowa 	 500.0 	 500.0 	 0.0 	  0.00
  Maryland 	 200.0 	 500.0 	 300.0 	  0.00
     Idaho 	 150.0 	 500.0 	 350.0 	  0.00
  Arkansas 	 500.0 	 500.0 	 0.0 	  0.00


In [16]:
# display flow constraint Information
print('DC \t\t LHS \t RHS \t Slack \t Dual')
for j in range(len(dc)):
    slack = LHS[j + len(factory) + len(customer)+8] - 0
    print('%10s \t %3.1f \t %3.1f \t %2.1f \t %5.2f' % (dc[j], LHS[j + len(factory) + len(customer)+8], 0, slack, flow_constr[j].dual_value()))

DC 		 LHS 	 RHS 	 Slack 	 Dual
      Iowa 	 0.0 	 0.0 	 0.0 	 42.50
  Maryland 	 0.0 	 0.0 	 0.0 	 40.00
     Idaho 	 0.0 	 0.0 	 0.0 	 40.00
  Arkansas 	 0.0 	 0.0 	 0.0 	 32.50


In [17]:
# increase demand in Butler by 100 and both suppliers by 100
in_processing_constr[0].SetUb(800)
out_processing_constr[0].SetUb(800)

In [18]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Total Cost = %.2f' % TotCost.Value())

Solution Status = 0
Number of variables = 24
Number of constraints = 18
Optimal Solution:
Total Cost = 79250.00


Expanding the Iowa DC would mean weekly total shipping cost went down to 79250 dollars, meaning the company saves 375 dollars per week, or 18750 in total in a year.
However annual amortized cost of this expansion is 40,000, exceeding the shipping cost that could be saved.
So I'd recommend not expanding the Iowa DC.